# State tomography
State tomography involves measuring a quantum state repeatedly in the bases given by `itertools.product(['X', 'Y', 'Z], repeat=n_qubits)`. From these measurements, we can reconstruct a density matrix $\rho$.

In [ ]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *

## Construct a state with a `Program`
We'll construct a two-qubit graph state by Hadamarding all qubits and then applying a controlled-Z operation across edges of our graph. In the two-qubit case, there's only one edge. 

In [ ]:
qubits = [0, 1]
program = Program()
for qubit in qubits:
    program += H(qubit)
program += CZ(qubits[0], qubits[1])
print(program)

## Construct a `TomographyExperiment` for state tomography
We can print this out to see the 16 measurements we will perform.

In [ ]:
from forest_qcvv.tomography import generate_state_tomography_experiment
experiment = generate_state_tomography_experiment(program=program, qubits=qubits)
print(experiment)

### Optional grouping
We can simultaneously estimate some of these observables

In [ ]:
from pyquil.operator_estimation import group_experiments
print(group_experiments(experiment))

## PyQuil will run the tomography programs

In [ ]:
from pyquil.operator_estimation import measure_observables

qc = get_qc('2q-pyqvm')
from forest_qcvv.compilation import basic_compile
qc.compiler.quil_to_native_quil = basic_compile

results = list(measure_observables(qc=qc, tomo_experiment=experiment, n_shots=100_000))
results

In [ ]:
# Half-way between conversion
from forest_qcvv.tomography import shim_pyquil_results_to_TomographyData
data = shim_pyquil_results_to_TomographyData(program=program, qubits=qubits, counts=100_000, results=results)

### We can look at a bunch of numbers...

In [ ]:
from forest_qcvv.tomography import linear_inv_state_estimate
rho = linear_inv_state_estimate(data).estimate.state_point_est
rho

In [ ]:
psi = (1/2) * np.array([1, 1, 1, -1])
rho_true = np.outer(psi, psi.T.conj())
rho_true

### Or visualize using Hinton plots

In [ ]:
from matplotlib import pyplot as plt
from forest_qcvv.plotting import hinton
fig, (ax1, ax2) = plt.subplots(1, 2)
hinton(rho_true, ax=ax1)
hinton(rho, ax=ax2)
ax1.set_title('Analytical')
ax2.set_title('Estimated')
fig.tight_layout()

### Matrix norm between true and estimated is low

In [ ]:
np.linalg.norm(rho - rho_true)

## Linear inversion estimate

In [ ]:
from forest_qcvv.tomography import linear_inv_state_estimate
rho = linear_inv_state_estimate(data).estimate.state_point_est

print(np.round(rho, 4))
print('Purity =', np.trace(rho @ rho))
hinton(rho)

## Maximum Liklihood Estimate (MLE) via diluted iterative method

In [ ]:
from forest_qcvv.tomography import iterative_mle_state_estimate
est_mle, status = iterative_mle_state_estimate(data, dilution=0.5)
rho = est_mle.estimate.state_point_est
print(np.around(rho, decimals=4))
print('Purity =', np.trace(rho @ rho))

## MLE with Max Entropy constraint

In [ ]:
est_max_ent, stat = iterative_mle_state_estimate(data, dilution=0.5, entropy_penalty=0.005)
rho = est_max_ent.estimate.state_point_est
print(np.around(rho, decimals=4))
print('Purity =', np.trace(rho @ rho))

## MLE with Hedging parameter

In [ ]:
est_hedging, stat = iterative_mle_state_estimate(data, dilution=0.5, beta=.61)
rho = est_hedging.estimate.state_point_est
print(np.around(rho, decimals=4))
print('Purity = ', np.trace(rho @ rho))

## Project an unphysical state to the closest physical state

In [ ]:
from forest_qcvv.tomography import project_density_matrix
rho_unphys = np.array([[1.0, 0], [0, -0.75]])
rho_phys = project_density_matrix(rho_unphys)

fig, (ax1, ax2) = plt.subplots(1, 2)
hinton(rho_unphys, ax=ax1)
hinton(rho_phys, ax=ax2)
ax1.set_title('Unphysical')
ax2.set_title('Physical projection')
fig.tight_layout()

In [ ]:
# Test the wizard method. Example from fig 1 of maximum likelihood minimum effort 
# https://doi.org/10.1103/PhysRevLett.108.070502

eigs = np.diag(np.array(list(reversed([3.0/5, 1.0/2, 7.0/20, 1.0/10, -11.0/20]))))
phys = project_density_matrix(eigs)
np.allclose(phys, np.diag([0, 0, 1.0/5, 7.0/20, 9.0/20]))

# Lightweight Bootstrap for functionals of the state

In [ ]:
import forest_qcvv.distance_measures as dm
from forest_qcvv.tomography import estimate_variance

In [ ]:
def my_mle_estimator(data):
    return iterative_mle_state_estimate(data, dilution=0.5, entropy_penalty=0.0, beta=0.0)[0]

**Purity**

In [ ]:
mle_est = estimate_variance(data, my_mle_estimator, dm.purity, n_resamples=40, project_to_physical=True)
lin_inv_est = estimate_variance(data, linear_inv_state_estimate, dm.purity, n_resamples=40, project_to_physical=True)
print(mle_est)
print(lin_inv_est)

**Fidelity**

In [ ]:
mle_est = estimate_variance(data, my_mle_estimator, dm.fidelity, target_state=rho_true, n_resamples=40, project_to_physical=True)
lin_inv_est = estimate_variance(data, linear_inv_state_estimate, dm.fidelity, target_state=rho_true, n_resamples=40, project_to_physical=True)
print(mle_est)
print(lin_inv_est)